In [ ]:
from simulator import *

import random
from utils import *
import numpy as np


def normalized_reciprocal(x):
    recip = 1/(x+1)  
    return recip / np.sum(recip)

def RandomReturn(stations_features):
    
    stations = []
    features = []
    
    for s,feature in stations_features.items():
        
        stations.append(s)
        features.append(feature)
        
    return random.choice(stations)

def NearestReturn(stations_features):
    stations = []
    dists = []
    
    for s,feature in stations_features.items():
        
        stations.append(s)
        dists.append(feature[2])
        
    min_index = np.argmin(dists)
    
    return stations[min_index]

def NearestPropReturn(stations_features):
    stations = []
    dists = []
    
    for s,feature in stations_features.items():
        
        stations.append(s)
        dists.append(feature[2])
        
    dists = np.array(dists)
    
    prop = normalized_reciprocal(dists)
    
    return np.random.choice(stations,p=prop)

def MostNeededReturn(stations_features):
    stations = []
    truck_nums = []
    
    for s,feature in stations_features.items():
        
        stations.append(s)
        truck_nums.append(feature[1])
        
    min_index = np.argmin(truck_nums)
    
    return stations[min_index]


def MostNeededProbReturn(stations_features):
    stations = []
    truck_nums = []
    
    for s,feature in stations_features.items():
        
        stations.append(s)
        truck_nums.append(feature[1])
        
    truck_nums = np.array(truck_nums)
    
    prop = normalized_reciprocal(truck_nums)
    
    return np.random.choice(stations,p=prop)

    


def RunSimulator(simulator:Environment,return_method):
        
        simulator.log('-----------------------------------------------------')
        simulator.log('System Begins')
        
        last_time = None
        last_state = None
        last_action = None
        
        sample_sequence = []
        
        while(1):
        
            stations_features,current_time,regu_reward,current_truck  = env.step()
            
            if last_time:
                
                sample_sequence.append((last_time,last_state,last_action,regu_reward))
            
            
            if stations_features == None:
                break
                
            last_state = stations_features
            last_time = current_time
                
            action = return_method(stations_features)
            simulator.take_action(action,current_truck)
            
            last_action = action
            
        return sample_sequence

metric_dict = {}

from tqdm import tqdm

env = Environment(10,1,running_day=1,start_date=datetime.datetime(2024,5,2))

for func in [RandomReturn,NearestReturn,NearestPropReturn,MostNeededProbReturn,MostNeededReturn]:
    
    
    func_name = func.__name__
    
    print(func_name)
    
    metric_list = []
    
    test_num = 10
    
    if func == NearestReturn or func == MostNeededReturn:
        test_num = 1
    
    for i in tqdm(range(test_num)):
        
        env.reset()
        
        
        RunSimulator(env,func)
        
        metric_list.append(env.count_time_per_fang())
        
    
    m = np.mean(metric_list)
    print(m)
    metric_dict[func_name] = m
    
    



RandomReturn


100%|██████████| 10/10 [00:30<00:00,  3.02s/it]


7.1489141550863256
NearestReturn


100%|██████████| 1/1 [00:03<00:00,  3.25s/it]


11.394253358231431
NearestPropReturn


...

7.149636046441678
MostNeededPropReturn


100%|██████████| 10/10 [00:33<00:00,  3.31s/it]


6.878613968597726
MostNeededReturn


100%|██████████| 1/1 [00:03<00:00,  3.16s/it]

8.999819527161161


In [2]:
metric_dict

{'RandomReturn': np.float64(7.1489141550863256),
 'NearestReturn': np.float64(11.394253358231431),
 'NearestPropReturn': np.float64(7.149636046441678),
 'MostNeededPropReturn': np.float64(6.878613968597726),
 'MostNeededReturn': np.float64(8.999819527161161)}

In [2]:
metric_dict

{'RandomReturn': np.float64(5.703771375029281),
 'NearestReturn': np.float64(12.116186460529399),
 'NearestPropReturn': np.float64(5.903849457328024),
 'MostNeededPropReturn': np.float64(5.66532365112829),
 'MostNeededReturn': np.float64(7.364566252830484)}

5.1-5.2

{'RandomReturn': np.float64(7.12410465511383),
 'NearestReturn': np.float64(18.433530733101897),
 'NearestPropReturn': np.float64(7.480319401970777),
 'MostNeededPropReturn': np.float64(6.916561331974177)}

5.2
{'RandomReturn': np.float64(7.1489141550863256),
 'NearestReturn': np.float64(11.394253358231431),
 'NearestPropReturn': np.float64(7.149636046441678),
 'MostNeededPropReturn': np.float64(6.878613968597726),
 'MostNeededReturn': np.float64(8.999819527161161)}

5.1

{'RandomReturn': np.float64(5.703771375029281),
 'NearestReturn': np.float64(12.116186460529399),
 'NearestPropReturn': np.float64(5.903849457328024),
 'MostNeededPropReturn': np.float64(5.66532365112829),
 'MostNeededReturn': np.float64(7.364566252830484)}